### Imports

In [78]:
from __future__ import division, print_function
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.inception import model_urls
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# import sparseconvnet.legacy as scn

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image, ImageChops

plt.ion()   # interactive mode
%matplotlib inline
%env TMP="/tmp"
%env JOBLIB_TEMP_FOLDER="/tmp"

env: TMP="/tmp"
env: JOBLIB_TEMP_FOLDER="/tmp"


In [79]:
def summary(input_size, model):
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split('.')[-1].split("'")[0]
            module_idx = len(summary)

            m_key = '%s-%i' % (class_name, module_idx+1)
            summary[m_key] = OrderedDict()
            summary[m_key]['input_shape'] = list(input[0].size())
            summary[m_key]['input_shape'][0] = -1
            summary[m_key]['output_shape'] = list(output.size())
            summary[m_key]['output_shape'][0] = -1

            params = 0
            if hasattr(module, 'weight'):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                if module.weight.requires_grad:
                    summary[m_key]['trainable'] = True
                else:
                    summary[m_key]['trainable'] = False
            if hasattr(module, 'bias'):
                params +=  torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]['nb_params'] = params

        if not isinstance(module, nn.Sequential) and \
           not isinstance(module, nn.ModuleList) and \
           not (module == model):
            hooks.append(module.register_forward_hook(hook))

    # check if there are multiple inputs to the network
    if isinstance(input_size[0], (list, tuple)):
        x = [Variable(torch.rand(1,*in_size)) for in_size in input_size]
    else:
        x = Variable(torch.rand(1,*input_size))

    # create properties
    summary = OrderedDict()
    hooks = []
    # register hook
    model.apply(register_hook)
    # make a forward pass
    model(x)
    # remove these hooks
    for h in hooks:
        h.remove()

    return summary

### Data

In [81]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TEST_DIR = '../data/boneage-validation-dataset/'
TRAIN_LABEL = '../data/train.csv'
TEST_LABEL = '../data/dataset_data_file-validation-gender-fa677c87-bd0d-44a1-92f3-f5966389798b.csv'
FILETYPE = '.png'
img_scale = 900
img_size = 840
n_channels = 1

use_gpu = torch.cuda.is_available()

### Dataset class

In [82]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, img_size, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype
        self.img_size = img_size
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        self.training = True if mode == 'train' else False
        self.predicting = True if mode == 'pred' else False

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
#         print(idx)
#         sample = 1.0
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name).convert('RGB')
#         print(img_name, img.size)
        
        # randomize thresholding
        if self.training:
            img = img.point(lambda x: x if x < np.random.randint(200,220) else 0)
        else:
            img = img.point(lambda x: x if x < 220 else 0)
            
        # resizing
#         print('PIL size:', img.size)
        img = np.asarray(img)
#         print('Np size: ',img.shape)
        if img.shape[1] > img.shape[0]:
#             print(img.shape)
            img = np.swapaxes(img,0,1)
    
        img = cv2.resize(img, (img_scale,img_scale))

#         img = cv2.resize(img, (img_scale,img_scale+210))
#         img = img[210:,:]
#         print(img.shape)

        # rotation
        if self.training:
            rot_ang = range(0,360,30)
            rot_ang = rot_ang[np.random.randint(0,len(rot_ang))]
    #         print(rot_ang)
            scale = np.random.uniform(0.8,1.2)
    #         print(scale)
            rot_mat = cv2.getRotationMatrix2D((img.shape[0]/2,img.shape[1]/2), rot_ang, scale);
            img = cv2.warpAffine(img, rot_mat, (img.shape[0],img.shape[1]))
        
        # randomize thresholding
        if self.training:
            thresh_block_size = np.random.randint(5,9) // 2 * 2 + 1
            thresh_param = np.random.randint(5,9)
        else:
            thresh_block_size = 7
            thresh_param = 7
        
        img_he = self.clahe.apply(cv2.convertScaleAbs(img[:,:,0]))
        img_mask = cv2.adaptiveThreshold(img_he, 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,
                                         thresh_block_size, thresh_param)
        img_ch = img_he * img_mask
        
        if n_channels == 1:
            img = img_ch
        elif n_channels == 3:
            for i in xrange(img.shape[2]):
                img[:,:,i] = img_ch
        else:
            print('\n Wrong number of channels \n')

        img = Image.fromarray(np.uint8(img))
#         print('Img:', img.size)
        
#         image = cv2.imread(img_name)[:,:,::-1]
        gender = self.labels.iloc[idx]['male'].astype(np.int32)
        pat_id = self.labels.iloc[idx]['id'].astype(np.int32)
        
        if not self.predicting:
            boneage = self.labels.iloc[idx]['boneage'].astype(np.float32)
            label = np.hstack((boneage, gender, pat_id))
        else:
            label = np.hstack((gender, pat_id))
#         print(label)

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [83]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.Scale(300),
#         transforms.CenterCrop(384),
        transforms.RandomSizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
#         transforms.Scale(300),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'pred': transforms.Compose([
#         transforms.Scale(300),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

### Data loaders

In [84]:
labels = pd.read_csv(TRAIN_LABEL, dtype=np.uint16)
bins = xrange(20)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [85]:
skf = StratifiedKFold(n_splits=7)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage', 'male')],
        'val': labels.loc[test, ('id', 'boneage', 'male')]
    }
    splits.append(fold_label)

/home/user/software/miniconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


In [86]:
fold_label = splits[0]

In [87]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, img_size, data_transforms[x], x)
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=24, shuffle=True, num_workers=24, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [88]:
for i_batch, sample in enumerate(dataloders['val']):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
#     for i in xrange(sample['image'].size()[0]):
#         img = sample['image'][i,0,:,:].numpy()
#         plt.figure()
#         plt.imshow(img, cmap=plt.cm.gray)
    if i_batch == 0:
        break

0 torch.Size([24, 1, 840, 840])
0 0.269442812354


### Training

In [89]:
model_urls['inception_v3_google'] = model_urls['inception_v3_google'].replace('https://', 'http://')

class Net(nn.Module):
    def __init__(self, input_shape=(n_channels, img_size, img_size)):
        super(Net, self).__init__()
        
#         # inception_v3
#         self.bn = nn.BatchNorm2d(input_shape[0])
#         resnet = models.inception_v3(pretrained=True)
        
#         num_ftrs = resnet.fc.in_features
#         resnet.fc = nn.Linear(num_ftrs, 1)
        
#         num_ftrs_aux = resnet.AuxLogits.fc.in_features
#         resnet.AuxLogits.fc = nn.Linear(num_ftrs_aux, 1)
        
#         self.resnet = resnet
        
#         # resnet
#         self.bn = nn.BatchNorm2d(input_shape[0])
#         resnet = models.resnet50(pretrained=True)
#         num_ftrs = resnet.fc.in_features
#         resnet.fc = nn.Linear(num_ftrs, 1)
#         self.resnet = resnet

        self.net = nn.Sequential(
            nn.Conv2d(input_shape[0], 192, 5, stride=3),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 160, 1, stride=1),
            nn.BatchNorm2d(160),
            nn.ReLU(inplace=True),
            nn.Conv2d(160, 96, 1, stride=1),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(3, stride=2, ceil_mode=True),
            nn.Dropout(),
            
            nn.Conv2d(96, 192, 5, stride=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            
            nn.AvgPool2d(3, stride=2, ceil_mode=True),
            nn.Dropout(),
        
            nn.Conv2d(192, 192, 3, stride=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 1, 1, stride=1),
            nn.BatchNorm2d(1),
            nn.ReLU(inplace=True)
                                 
#             nn.AvgPool2d(12, stride=12, ceil_mode=True)
        )
        self.boneage_clf = nn.Sequential(
            nn.Dropout(),
            nn.Linear(12*12, 72),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(72, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1)
        )
        self.gender_clf = nn.Sequential(
            nn.Dropout(),
            nn.Linear(12*12, 72),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(72, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1)
        )
        
#         resnet.avgpool = nn.AdaptiveAvgPool2d(2)
#         num_ftrs = net.fc.in_features
#         net.fc = nn.Linear(num_ftrs, 1)
#         self.net = net

    def forward(self, x):
#         x = self.net(self.bn(x))
        x = self.net(x)
#         print(x.size())
        x = x.view(-1, 12*12)
        g_x = self.gender_clf(x)
        ba_x = self.boneage_clf(x)
#         x = F.relu(self.lin128(F.dropout(x)))
#         x = self.lin1(F.dropout(x))
#         x = self.resnet(self.bn(x))
#         return ba_x
        return ba_x, g_x
    
model = Net([n_channels, img_size, img_size])
    
# print(summary([3, 640, 640], model))
print(model)

if use_gpu:
    model.cuda()

Net (
  (net): Sequential (
    (0): Conv2d(1, 192, kernel_size=(5, 5), stride=(3, 3))
    (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): Conv2d(192, 160, kernel_size=(1, 1), stride=(1, 1))
    (4): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): Conv2d(160, 96, kernel_size=(1, 1), stride=(1, 1))
    (7): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (10): Dropout (p = 0.5)
    (11): Conv2d(96, 192, kernel_size=(5, 5), stride=(2, 2))
    (12): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (13): ReLU (inplace)
    (14): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (15): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (16): ReLU (inplace)
    (17): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (18): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (1

In [90]:
lr = 0.01
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [91]:
def ccc(x, y):
#     print('X:', x)
    mean_x = torch.mean(x)
#     print('Mean x:', mean_x)
    mean_y = torch.mean(y)
#     print('Mean y:', mean_y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    xn = torch.norm(xm, 2)
#     print('Xn:', xn)
    yn = torch.norm(ym, 2)
#     print('Yn:', yn)
    xydot = ym.dot(xm.view(-1))
#     print('XYdot:', xydot)

    ccc_num = 2 * xydot / xm.size()[0]
#     print('CCC num:', ccc_num)
    ccc_den = xn + yn + torch.pow(mean_x.sub(mean_y), 2)
#     print('CCC den:', ccc_den)

    ccc = ccc_num / ccc_den
    return ccc
    

def train(epoch, train_loader):
    exp_lr_scheduler.step()
    model.train()
    train_boneage_loss = 0
    train_gender_loss = 0
    train_loss = 0
    train_ccc = 0
    num_samp = len(train_loader.dataset)
    
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        
#         ba_output = model(data)
        
        # 2 heads MTL output
        ba_output, g_output = model(data)
#         print(output1)
#         print(output2)
        
#         for i in xrange(output.size[0]):
#             if output[i,1] > 0.5:
#                 output[i,0] = (output[i,0] // 6) * 6
#             else:
#                 output[i,0] = (output[i,0] // 2) * 2

#         boneage_out = torch.round(output[:,0].div(6)).mul(6)
#         boneage_out = output[:,0]

#         boneage_loss = F.l1_loss(output.view(-1), target[:,0])
#         train_boneage_loss += boneage_loss.data[0]
        boneage_loss = F.smooth_l1_loss(ba_output.view(-1), target[:,0])
        train_boneage_loss += boneage_loss.data[0]
        
        gender_loss = F.binary_cross_entropy_with_logits(g_output.view(-1), target[:,1])
        train_gender_loss += gender_loss.data[0]
        
        loss = 0.75 * boneage_loss + 0.25 * gender_loss
#         loss = boneage_loss
#         loss = gender_loss
#         loss = F.l1_loss(output1.view(-1), target[:,0]) + F.l1_loss(output2.view(-1), target[:,0])
#         loss = 0.8 * F.l1_loss(output.view(-1), target[:,0]) + 0.2 * F.binary_cross_entropy(F.sigmoid(output.view(-1)), target[:,1])
#         loss = F.l1_loss(output.view(-1), target[:,0])
        
        train_loss += loss.data[0]
#         c = ccc(boneage_out, target[:,0])
        c = ccc(ba_output, target[:,0])
        train_ccc += c.data.cpu().numpy()[0]
        
        loss.backward()
        optimizer.step()
#         if batch_idx % (1000 // len(target)) == 0:
    print('Train ep: {} [{}/{} ({:.0f}%)]\tBoneAge Loss: {:.4f}\tGender Loss: {:.4f}\t Loss: {:.4f}\tCCC: {:.4f}'.format(
        epoch, batch_idx * len(data), num_samp,
        100. * batch_idx / num_samp, train_boneage_loss / num_samp,
        train_gender_loss / num_samp, train_loss / num_samp,
        train_ccc / num_samp))
    
#     print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.4f}\tCCC: {:.4f}'.format(
#         epoch, batch_idx * len(data), num_samp,
#         100. * batch_idx / num_samp, train_loss / num_samp, train_ccc / num_samp))

def test(test_loader):
    model.eval()
    test_avg_gender = 0
    test_boneage_loss = 0
    test_boneage_l1_loss = 0
    test_boneage_round_loss = 0
    test_boneage_round_loss_2 = 0
    test_boneage_round_loss_6 = 0
    test_gender_loss = 0
    test_loss = 0
    test_ccc = 0
    num_samp = len(test_loader.dataset)
    for sample in test_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)

#         test_avg_gender += sample['label'][:,1].sum()
#         ba_output = model(data)
        
        # 2 heads MTL output
        ba_output, g_output = model(data)
#         print(ba_output)
        
#         print(output.size())
#         boneage_out = torch.round(output[:,0].div(6)).mul(6)
#         boneage_out = output[:,0]
#         print(boneage_out)
#         print(target[:,0])
#         boneage_loss = F.l1_loss(output.view(-1), target[:,0])

        boneage_loss = F.smooth_l1_loss(ba_output.view(-1), target[:,0])
        boneage_l1_loss = F.l1_loss(ba_output.view(-1), target[:,0])
# #         print(boneage_loss)
        boneage_round_loss = F.l1_loss(torch.round(ba_output.view(-1)), target[:,0])
# #         print(boneage_round_loss)
        boneage_round_loss_2 = F.l1_loss(torch.round(ba_output.view(-1).div(2)).mul(2), target[:,0])
# #         print(boneage_round_loss_2)
        boneage_round_loss_6 = F.l1_loss(torch.round(ba_output.view(-1).div(6)).mul(6), target[:,0])
# #         print(boneage_round_loss_6)
        
        test_boneage_loss += boneage_loss.data[0]
        test_boneage_l1_loss += boneage_l1_loss.data[0]
        test_boneage_round_loss += boneage_round_loss.data[0]
        test_boneage_round_loss_2 += boneage_round_loss_2.data[0]
        test_boneage_round_loss_6 += boneage_round_loss_6.data[0]
    
#         print(test_boneage_loss)
        gender_loss = F.binary_cross_entropy_with_logits(g_output.view(-1), target[:,1])
        test_gender_loss += gender_loss.data[0]

        loss = 0.75 * boneage_loss + 0.25 * gender_loss
#         loss = boneage_loss
#         loss = 0.8 * F.l1_loss(output.view(-1), target[:,0]) + 0.2 * F.binary_cross_entropy(F.sigmoid(output.view(-1)), target[:,1])
#         loss = F.l1_loss(output.view(-1), target[:,0])

        test_loss += loss.data[0]
#         c = ccc(boneage_out, target[:,0])
        c = ccc(ba_output, target[:,0])
        test_ccc += c.data.cpu().numpy()[0]

    print('\nTest set: BoneAge loss: {:.4f}, Gender loss: {:.4f}, loss: {:.4f}, CCC: {:.4f}'.format(
        test_boneage_loss / num_samp, test_gender_loss / num_samp,
        test_loss / num_samp, test_ccc / num_samp))
    print('\nTest set: L1 loss: {:.4f},Round loss: {:.4f}, Round2 loss: {:.4f}, Round6: {:.4f}, Gender: {:.4f}\n'.format(
        test_boneage_l1_loss / num_samp, test_boneage_round_loss / num_samp, test_boneage_round_loss_2 / num_samp,
        test_boneage_round_loss_6 / num_samp, test_avg_gender / num_samp))
    
#     print('\nTest set: Avg loss: {:.4f} CCC: {:.4f}\n'.format(
#         test_loss / num_samp, test_ccc / num_samp))

In [92]:
for epoch in range(1, 60 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

RuntimeError: invalid argument 2: size '[-1 x 144]' is invalid for input of with 6144 elements at /opt/conda/conda-bld/pytorch_1502006348621/work/torch/lib/TH/THStorage.c:37

In [16]:
lr = 0.001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

for epoch in range(61, 61+60 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train ep: 61 [3835/9453 (3%)]	BoneAge Loss: 0.9088	Gender Loss: 0.0216	 Loss: 0.6870	CCC: 0.0578

Test set: BoneAge loss: 1.4785, Gender loss: 0.0216, loss: 1.1143, CCC: 0.0071

Test set: L1 loss: 1.4941,Round loss: 1.4937, Round2 loss: 1.4937, Round6: 1.4855, Gender: 0.0000

Train ep: 62 [3835/9453 (3%)]	BoneAge Loss: 0.8896	Gender Loss: 0.0216	 Loss: 0.6726	CCC: 0.0632

Test set: BoneAge loss: 1.5119, Gender loss: 0.0216, loss: 1.1394, CCC: 0.0089

Test set: L1 loss: 1.5276,Round loss: 1.5273, Round2 loss: 1.5289, Round6: 1.5372, Gender: 0.0000

Train ep: 63 [3835/9453 (3%)]	BoneAge Loss: 0.8754	Gender Loss: 0.0216	 Loss: 0.6619	CCC: 0.0664

Test set: BoneAge loss: 1.3223, Gender loss: 0.0216, loss: 0.9972, CCC: 0.0113

Test set: L1 loss: 1.3380,Round loss: 1.3382, Round2 loss: 1.3377, Round6: 1.3378, Gender: 0.0000

Train ep: 64 [3835/9453 (3%)]	BoneAge Loss: 0.8781	Gender Loss: 0.0216	 Loss: 0.6640	CCC: 0.0672

Test set: BoneAge loss: 1.3460, Gender loss: 0.0216, loss: 1.0149, CCC:

Train ep: 91 [3835/9453 (3%)]	BoneAge Loss: 0.8520	Gender Loss: 0.0216	 Loss: 0.6444	CCC: 0.0740

Test set: BoneAge loss: 1.2997, Gender loss: 0.0216, loss: 0.9802, CCC: 0.0172

Test set: L1 loss: 1.3153,Round loss: 1.3150, Round2 loss: 1.3158, Round6: 1.3145, Gender: 0.0000

Train ep: 92 [3835/9453 (3%)]	BoneAge Loss: 0.8496	Gender Loss: 0.0216	 Loss: 0.6426	CCC: 0.0750

Test set: BoneAge loss: 1.3096, Gender loss: 0.0216, loss: 0.9876, CCC: 0.0173

Test set: L1 loss: 1.3252,Round loss: 1.3254, Round2 loss: 1.3259, Round6: 1.3246, Gender: 0.0000

Train ep: 93 [3835/9453 (3%)]	BoneAge Loss: 0.8385	Gender Loss: 0.0216	 Loss: 0.6343	CCC: 0.0772

Test set: BoneAge loss: 1.3094, Gender loss: 0.0216, loss: 0.9875, CCC: 0.0171

Test set: L1 loss: 1.3250,Round loss: 1.3255, Round2 loss: 1.3251, Round6: 1.3229, Gender: 0.0000

Train ep: 94 [3835/9453 (3%)]	BoneAge Loss: 0.8468	Gender Loss: 0.0216	 Loss: 0.6405	CCC: 0.0756

Test set: BoneAge loss: 1.3180, Gender loss: 0.0216, loss: 0.9939, CCC:

Train ep: 121 [3835/9453 (3%)]	BoneAge Loss: 0.8530	Gender Loss: 0.0216	 Loss: 0.6452	CCC: 0.0739

Test set: BoneAge loss: 1.3025, Gender loss: 0.0216, loss: 0.9823, CCC: 0.0163

Test set: L1 loss: 1.3181,Round loss: 1.3182, Round2 loss: 1.3186, Round6: 1.3170, Gender: 0.0000



In [17]:
torch.save(model.state_dict(), '../models/10-3-0845-nin-0fold-085-131.pth')

In [ ]:
lr = 0.001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

for epoch in range(121, 121+60 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train ep: 121 [3835/9453 (3%)]	BoneAge Loss: 0.8627	Gender Loss: 0.0216	 Loss: 0.6525	CCC: 0.0720

Test set: BoneAge loss: 1.3126, Gender loss: 0.0216, loss: 0.9899, CCC: 0.0205

Test set: L1 loss: 1.3282,Round loss: 1.3278, Round2 loss: 1.3278, Round6: 1.3248, Gender: 0.0000

Train ep: 122 [3835/9453 (3%)]	BoneAge Loss: 0.8526	Gender Loss: 0.0216	 Loss: 0.6449	CCC: 0.0749

Test set: BoneAge loss: 1.4653, Gender loss: 0.0216, loss: 1.1043, CCC: 0.0212

Test set: L1 loss: 1.4809,Round loss: 1.4806, Round2 loss: 1.4825, Round6: 1.4846, Gender: 0.0000



In [19]:
test(dataloders['val'])


Test set: BoneAge loss: 1.3295, Gender loss: 0.0216, loss: 1.0025, CCC: 0.0158

Test set: L1 loss: 1.3451,Round loss: 1.3451, Round2 loss: 1.3452, Round6: 1.3464, Gender: 0.0000



In [20]:
lr = 0.01
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

for epoch in range(150, 150+20 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train ep: 150 [3835/9453 (3%)]	BoneAge Loss: 0.9657	Gender Loss: 0.0216	 Loss: 0.7297	CCC: 0.0401

Test set: BoneAge loss: 1.2380, Gender loss: 0.0216, loss: 0.9339, CCC: 0.0132

Test set: L1 loss: 1.2536,Round loss: 1.2531, Round2 loss: 1.2534, Round6: 1.2542, Gender: 0.0000

Train ep: 151 [3835/9453 (3%)]	BoneAge Loss: 0.9573	Gender Loss: 0.0216	 Loss: 0.7233	CCC: 0.0400

Test set: BoneAge loss: 1.3258, Gender loss: 0.0216, loss: 0.9998, CCC: 0.0051

Test set: L1 loss: 1.3414,Round loss: 1.3422, Round2 loss: 1.3418, Round6: 1.3219, Gender: 0.0000

Train ep: 152 [3835/9453 (3%)]	BoneAge Loss: 0.9457	Gender Loss: 0.0216	 Loss: 0.7147	CCC: 0.0399

Test set: BoneAge loss: 1.4564, Gender loss: 0.0216, loss: 1.0977, CCC: 0.0006

Test set: L1 loss: 1.4718,Round loss: 1.4720, Round2 loss: 1.4720, Round6: 1.4504, Gender: 0.0000

Train ep: 153 [3835/9453 (3%)]	BoneAge Loss: 0.8936	Gender Loss: 0.0216	 Loss: 0.6756	CCC: 0.0558

Test set: BoneAge loss: 1.4139, Gender loss: 0.0216, loss: 1.0658, 

In [21]:
lr = 0.001
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

for epoch in range(170, 170+20 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train ep: 170 [3835/9453 (3%)]	BoneAge Loss: 0.8422	Gender Loss: 0.0216	 Loss: 0.6370	CCC: 0.0715

Test set: BoneAge loss: 1.3370, Gender loss: 0.0216, loss: 1.0082, CCC: 0.0159

Test set: L1 loss: 1.3526,Round loss: 1.3526, Round2 loss: 1.3526, Round6: 1.3497, Gender: 0.0000

Train ep: 171 [3835/9453 (3%)]	BoneAge Loss: 0.8348	Gender Loss: 0.0216	 Loss: 0.6315	CCC: 0.0782

Test set: BoneAge loss: 1.2850, Gender loss: 0.0216, loss: 0.9692, CCC: 0.0144

Test set: L1 loss: 1.3006,Round loss: 1.3006, Round2 loss: 1.3005, Round6: 1.2997, Gender: 0.0000

Train ep: 172 [3835/9453 (3%)]	BoneAge Loss: 0.8313	Gender Loss: 0.0216	 Loss: 0.6289	CCC: 0.0777

Test set: BoneAge loss: 1.4696, Gender loss: 0.0216, loss: 1.1076, CCC: 0.0090

Test set: L1 loss: 1.4852,Round loss: 1.4857, Round2 loss: 1.4857, Round6: 1.4863, Gender: 0.0000

Train ep: 173 [3835/9453 (3%)]	BoneAge Loss: 0.8251	Gender Loss: 0.0216	 Loss: 0.6242	CCC: 0.0781

Test set: BoneAge loss: 1.3676, Gender loss: 0.0216, loss: 1.0311, 

In [22]:
lr = 0.01
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

for epoch in range(190, 190+20 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train ep: 190 [3835/9453 (3%)]	BoneAge Loss: 0.9402	Gender Loss: 0.0216	 Loss: 0.7106	CCC: 0.0439

Test set: BoneAge loss: 1.3529, Gender loss: 0.0216, loss: 1.0201, CCC: 0.0061

Test set: L1 loss: 1.3685,Round loss: 1.3652, Round2 loss: 1.3653, Round6: 1.3446, Gender: 0.0000

Train ep: 191 [3835/9453 (3%)]	BoneAge Loss: 0.9251	Gender Loss: 0.0216	 Loss: 0.6992	CCC: 0.0438

Test set: BoneAge loss: 1.4186, Gender loss: 0.0216, loss: 1.0693, CCC: 0.0072

Test set: L1 loss: 1.4342,Round loss: 1.4336, Round2 loss: 1.4334, Round6: 1.4339, Gender: 0.0000

Train ep: 192 [3835/9453 (3%)]	BoneAge Loss: 0.8672	Gender Loss: 0.0216	 Loss: 0.6558	CCC: 0.0591

Test set: BoneAge loss: 1.1529, Gender loss: 0.0216, loss: 0.8701, CCC: 0.0320

Test set: L1 loss: 1.1685,Round loss: 1.1685, Round2 loss: 1.1689, Round6: 1.1681, Gender: 0.0000

Train ep: 193 [3835/9453 (3%)]	BoneAge Loss: 0.8417	Gender Loss: 0.0216	 Loss: 0.6367	CCC: 0.0701

Test set: BoneAge loss: 1.1581, Gender loss: 0.0216, loss: 0.8740, 

In [23]:
torch.save(model.state_dict(), '../models/10-3-0300-nin-0fold-083-119.pth')

In [24]:
lr = 0.01
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(210, 210+8 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train ep: 210 [3835/9453 (3%)]	BoneAge Loss: 0.8957	Gender Loss: 0.0216	 Loss: 0.6772	CCC: 0.0560

Test set: BoneAge loss: 1.4372, Gender loss: 0.0216, loss: 1.0833, CCC: 0.0082

Test set: L1 loss: 1.4527,Round loss: 1.4553, Round2 loss: 1.4562, Round6: 1.4587, Gender: 0.0000

Train ep: 211 [3835/9453 (3%)]	BoneAge Loss: 0.8634	Gender Loss: 0.0216	 Loss: 0.6530	CCC: 0.0599

Test set: BoneAge loss: 1.1618, Gender loss: 0.0216, loss: 0.8768, CCC: 0.0306

Test set: L1 loss: 1.1774,Round loss: 1.1779, Round2 loss: 1.1763, Round6: 1.1730, Gender: 0.0000

Train ep: 212 [3835/9453 (3%)]	BoneAge Loss: 0.8407	Gender Loss: 0.0216	 Loss: 0.6359	CCC: 0.0660

Test set: BoneAge loss: 1.1441, Gender loss: 0.0216, loss: 0.8635, CCC: 0.0286

Test set: L1 loss: 1.1597,Round loss: 1.1595, Round2 loss: 1.1603, Round6: 1.1622, Gender: 0.0000

Train ep: 213 [3835/9453 (3%)]	BoneAge Loss: 0.8402	Gender Loss: 0.0216	 Loss: 0.6355	CCC: 0.0664

Test set: BoneAge loss: 1.1426, Gender loss: 0.0216, loss: 0.8623, 

In [27]:
torch.save(model.state_dict(), '../models/10-3-0320-nin-0fold-083-116.pth')

### Prediction

In [13]:
model.load_state_dict(torch.load('../models/10-3-0320-nin-0fold-083-116.pth'))

In [54]:
def predict(predict_loader):
    model.eval()
    pred_gender_loss = 0
    predictions = []
    
    num_samp = len(predict_loader.dataset)
    for sample in predict_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)

        ba_output, g_output = model(data)
    
#         print(test_boneage_loss)
        gender_loss = F.binary_cross_entropy_with_logits(g_output.view(-1), target[:,0])
        pred_gender_loss += gender_loss.data[0]
        
        pred_labels = target[:,1].data.cpu().numpy()
#         print(pred_labels.shape)
        pred_bonage = ba_output.view(-1).data.cpu().numpy()
#         print(pred_bonage.shape)
        
        predictions.append(np.column_stack((pred_labels, pred_bonage)))

    print('\nPrediction Gender loss: {:.4f}'.format(pred_gender_loss / num_samp))
    
    return predictions

In [15]:
pred_labels = pd.read_csv(TEST_LABEL, dtype=int)
pred_labels.describe()

,id,male
count,1425.000000,1425.000000
mean,8678.919298,0.542456
std,4123.833964,0.498369
min,1386.000000,0.000000
25%,5196.000000,0.000000
50%,8784.000000,1.000000
75%,12147.000000,1.000000
max,15612.000000,1.000000


In [17]:
pred_image_dataset = BoneAgeDataset(TEST_DIR, pred_labels, FILETYPE, img_size, data_transforms['pred'], 'pred')
pred_dataloder = torch.utils.data.DataLoader(pred_image_dataset, batch_size=32, shuffle=True, num_workers=24,
                                             pin_memory=True)
pred_dataset_size = len(pred_image_dataset)

In [33]:
for i_batch, sample in enumerate(pred_dataloder):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
#     for i in xrange(sample['image'].size()[0]):
#         img = sample['image'][i,0,:,:].numpy()
#         plt.figure()
#         plt.imshow(img, cmap=plt.cm.gray)
    if i_batch == 2:
        break

0 torch.Size([32, 1, 640, 640])
0 0.238465481364
1 torch.Size([32, 1, 640, 640])
1 0.26067258119
2 torch.Size([32, 1, 640, 640])
2 0.252933594625


In [55]:
preds = predict(pred_dataloder)


Prediction Gender loss: 0.0218


In [74]:
all_preds_df = pd.DataFrame(np.vstack(tuple(preds)), columns=('id', 'boneage')).sort_values('id')
all_preds_df['id'] = all_preds_df['id'].astype(int)

In [75]:
all_preds_df.head()

,id,boneage
37,1386,86.842384
524,1392,79.800110
226,1397,137.686981
1337,1401,98.267731
927,1410,79.800110


In [76]:
all_preds_df.tail()

,id,boneage
120,15592,95.326370
1395,15601,153.793701
1039,15607,172.908478
74,15611,147.293030
1195,15612,141.046539


In [77]:
all_preds_df.to_csv('../sub/10-3-0707-nin-0fold-083-116.csv', index=False)